# update-yearly-stats

Updates https://www.data.gouv.fr/fr/datasets/statistiques-de-consultation-de-data-gouv-fr/#_ every year with data from previous year.

In [ ]:
import requests
import pandas as pd

In [ ]:
import os
from datetime import datetime
default_year = datetime.now().year - 1
year = os.getenv('NB_AUDIENCE_YEAR', default_year)
year

In [ ]:
def get_months(site_id, year):

    endpoint = 'https://stats.data.gouv.fr/'
    
    params = {
        'idSite': site_id,
        'module': 'API',
        'method': 'API.get',
        'format': 'json',
        'period': 'day',
        'date': '{0}-01-01,{0}-12-31'.format(year),
    }
        
    r = requests.get(endpoint, params=params)
    
    df = pd.DataFrame(r.json()).transpose()

    return df

df = get_months(109, year)

df

In [ ]:
csv_filename = "{}-days.csv".format(year)
df.to_csv(csv_filename)

In [ ]:
apikey = os.environ['NB_IRVE_APIKEY']
dataset_id = "5a22644f88ee3848529af925"

In [ ]:
# send the results to the portal
import requests

url = 'https://www.data.gouv.fr/api/1/datasets/%s' % dataset_id

with open(csv_filename, 'rb') as csvfile:
    files = {'file': (csv_filename, csvfile.read())}

# upload file
r = requests.post('%s/upload/' % url, files=files, headers={'x-api-key': apikey})
assert r.status_code == 201

# update resource's metadata
resource = r.json()
resource['title'] = "Statistiques de consultation pour l'année {}".format(year)
r = requests.put('%s/resources/%s/' % (url, resource['id']), json=resource, headers={'x-api-key': apikey})
assert r.status_code == 200
print(r.json())